In [4]:
import pandas as pd


btc = pd.read_csv('/Users/hatim/Desktop/Applied Forecasting/Final Project/Algo Trading/btc_daily.csv', index_col=0, parse_dates=True)
btc

,Open,High,Low,Close,Volume
2015-07-20,277.98,280.00,277.37,280.00,782.883420
2015-07-21,279.96,281.27,276.85,277.32,4943.559434
2015-07-22,277.33,278.54,275.01,277.89,4687.909383
2015-07-23,277.96,279.75,276.28,277.39,5306.919575
2015-07-24,277.23,291.52,276.43,289.12,7362.469083
...,...,...,...,...,...
2025-04-04,83178.68,84720.67,81643.54,83860.16,14593.030929
2025-04-05,83859.78,84238.35,82346.61,83498.25,2836.429207
2025-04-06,83505.88,83773.58,77058.99,78370.75,11014.859477
2025-04-07,78370.15,81223.67,74420.69,79140.01,26706.529308


# Finading the optimal trading strategy
- 1 will be buy
- -1 will be sell
- 0 will be hold

The goal is to maximize return while considering a trasaction costs

In [ ]:
# detect local maximums


dtype('<M8[ns]')